In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
#import modules from files in a parallel directory "direct_regression"


from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    config['mask_location'] + 'masks/aim3/neurosynth/',
    config['mask_location'] + 'masks/aim3/'
]

signature_locations = [
    config['mask_location'] + 'signatures/aim3/'
]

roi_df = get_mask_df_from_mask_locations(mask_locations)
signature_df = get_mask_df_from_mask_locations(signature_locations)
roi_df['image_type'] = 'mask'
signature_df['image_type'] = 'signature'

#combine the two dfs
roi_df = pd.concat([roi_df, signature_df])
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'fmriprep_dir': '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep_2022/', 'nii_raw_path': 'sub-DEV*/ses-wave*/func/s6_sub-DEV*_ses-wave*_task-SST_acq-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii', 'mask_location': '/

## SST

In [ ]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|finger|motor|response inhibition')]

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)
train_betas_with_data['wave']=1

#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])

beta_name_list = [
    'CorrectGo',
    'CorrectStop',
    'FailedStop',
    #'Cue',
    #'FailedGo'
    ]

#get the ROI data
roi_data_sst_conditions = get_roi_data_for_l2_betas(betas_with_paths, beta_name_list, sst_roi_df)



In [ ]:

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)
train_betas_with_data['wave']=1

#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])

beta_name_list = ['CorrectGoFollowingFailedStop']

#get the ROI data
roi_data_sst_posterror_conditions = get_roi_data_for_l2_betas(betas_with_paths, beta_name_list, sst_roi_df)



In [6]:
roi_data_sst=pd.concat([roi_data_sst_conditions, roi_data_sst_posterror_conditions], axis=1)

In [7]:
roi_data_sst.to_csv(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.csv')

### WTP

In [ ]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
wtp_mask_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|value')]


## WTP
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False,
    task='WTP'
).copy()
train_betas_with_data['wave']=1



#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fname_list_for_beta_dirs(train_betas_with_data[0:5])

#from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas

beta_name_list = [
    'liked'
    ]

#get the ROI data
roi_data_wtp = get_roi_data_for_multirun_l2_betas(betas_with_paths, condition_list = beta_name_list, mask_df = wtp_mask_df)

roi_data_wtp.to_csv(config['dropbox_data_dir'] + '/subject_wtp_avg_roi_data_raw.csv')


### ROC

In [9]:
roi_df

,mask_label,mask_path
0,reward_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
1,response inhibition_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
2,finger movements_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
3,reappraisal_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
4,abstract_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
5,attention network_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
6,motor control_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
7,value_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
8,striatum_joint_mask,/Users/benjaminsmith/Google Drive/oregon/data/...
9,koban_kober_craving_wmapN99_boot10K_02-May-2022,/Users/benjaminsmith/Google Drive/oregon/data/...


In [10]:
roc_mask_df = roi_df.loc[roi_df['mask_label'].str.contains('reappraisal|abstract|attention|striatum|craving|regulate')]


train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/ROC/wave1/conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False,
    task='ROC'
).copy()
train_betas_with_data['wave']=1



#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fname_list_for_beta_dirs(train_betas_with_data[0:5])



#from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas

beta_name_list = [
    'lookCrave',
    'reappraiseCrave'
    ]

#get the ROI data
roi_data_roc = get_roi_data_for_multirun_l2_betas(betas_with_paths, condition_list = beta_name_list, mask_df = roc_mask_df)

roi_data_roc.to_csv(config['dropbox_data_dir'] + '/subject_roc_avg_roi_data_raw.csv')


loaded 275 rows from data_by_ppt.csv
247 subjects with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
0 subjects excludeable for missing scan data.
247 subjects remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
247 subjects remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 279
beta paths after exclusion via nsc_subject_exclusions and the provision useable_dev_id list: 241
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/ROC/wave1/conditions/sub-DEV004/


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_train_betas_with_data.sort_values('subject_id', inplace=True)
/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beta_dir_df['spm_l2_targeted_contrast_filepath'] = None


/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/ROC/wave1/conditions/sub-DEV005/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/ROC/wave1/conditions/sub-DEV006/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/ROC/wave1/conditions/sub-DEV008/
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/ROC/wave1/conditions/sub-DEV010/
reappraisal_association-test_z_FDR_0.01
producing the vector for this mask/series...
abstract_association-test_z_FDR_0.01
producing the vector for this mask/series...
attention network_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_mask
producing the vector for this mask/series...
koban_kober_craving_wmapN99_boot10K_02-May-2022
producing the vector for this mask/series...
motor_control_striatum_joint_mask
producing the vector for this mask/series...
univariate_regulate_look
producing the vector for this mask/s

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.0248494734458682
DEV004, 1, lookCrave, abstract_association-test_z_FDR_0.01


...

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.2857643237048709
DEV010, 1, reappraiseCrave, motor_control_striatum_joint_mask


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.0013474610340700624
DEV010, 1, reappraiseCrave, univariate_regulate_look
proportion of image active: 0.25819774776231824


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


In [ ]:
#save
with open(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.pkl', 'wb') as handle:
    pickle.dump(roi_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

